# CPU Monitoring in Jupyter Notebooks

This notebook demonstrates how to use PtDAlgorithms' CPU monitoring features in Jupyter.

## Features

- **Cell Magic `%%usage`**: Monitor CPU during cell execution
- **Context Manager**: Use `with CPUMonitor()` for explicit control
- **Auto-width**: Automatically fits notebook width
- **Per-core bars**: Beautiful widget-based progress bars
- **SLURM-aware**: Detects allocated nodes and CPUs
- **Summary stats**: Shows mean/min/max usage after completion

In [ ]:
import time
import numpy as np
import ptdalgorithms as pta

print("PtDAlgorithms CPU Monitoring Loaded!")
print("Available features:")
print("  - %%usage cell magic (auto-registered)")
print("  - CPUMonitor context manager")
print("  - monitor_cpu decorator")

## Example 1: Using `%%usage` Cell Magic

The easiest way to monitor CPU usage is with the `%%usage` cell magic.
Simply add `%%usage` at the top of any cell!

In [ ]:
%%usage

# Simulate some computation
print("Running computation...")
for i in range(5):
    x = sum(range(10_000_000))
    time.sleep(0.5)
print("Done!")

## Example 2: Custom Width and Update Interval

You can customize the display width and update frequency:

In [ ]:
%%usage --width 100 --interval 0.25

# More frequent updates (4 times per second)
print("Running with custom settings...")
for i in range(10):
    x = sum(range(5_000_000))
    time.sleep(0.3)
print("Complete!")

## Example 3: Context Manager

For more control, use the `CPUMonitor` context manager:

In [ ]:
print("Using context manager...")

with pta.CPUMonitor(update_interval=0.5):
    # Your computation here
    for i in range(5):
        result = np.linalg.eigvals(np.random.randn(500, 500))
        time.sleep(0.5)

print("Monitoring complete!")

## Example 4: NumPy Operations

Monitor CPU usage during real computational work:

In [ ]:
%%usage

# Matrix operations
n = 2000
print(f"Creating {n}x{n} matrices...")
A = np.random.randn(n, n)
B = np.random.randn(n, n)

print("Matrix multiplication...")
C = np.dot(A, B)

print("Computing eigenvalues...")
eigenvalues = np.linalg.eigvals(C[:500, :500])

print(f"Computed {len(eigenvalues)} eigenvalues")

## Example 5: PtDAlgorithms Graph Operations

Monitor CPU usage during phase-type distribution computations:

In [ ]:
# Create a phase-type distribution
g = pta.Graph(1)
start = g.starting_vertex()

# Create vertices
v0 = g.find_or_create_vertex([0])
v1 = g.find_or_create_vertex([1])
v2 = g.find_or_create_vertex([2])

# Add transitions
start.add_edge(v0, 1.0)
v0.add_edge(v1, 2.0)
v1.add_edge(v2, 1.5)

g.normalize()

print(f"Created graph with {g.vertices_length()} vertices")

In [ ]:
%%usage

# Evaluate PDF with monitoring
print("Evaluating PDF at 50,000 time points...")
times = np.linspace(0.1, 10.0, 50_000)
pdf_values = g.pdf_batch(times)

print(f"PDF computed!")
print(f"  Max value: {np.max(pdf_values):.4f}")
print(f"  Mean value: {np.mean(pdf_values):.4f}")

## Example 6: Parallel Computation with JAX

If JAX is available, monitor parallel computations:

In [ ]:
try:
    # Initialize parallel configuration
    config = pta.init_parallel()
    print(f"Parallel config: {config.device_count} devices, strategy={config.strategy}")
except Exception as e:
    print(f"Could not initialize parallel config: {e}")

In [ ]:
%%usage

# Run computation with monitoring
print("Running parallel computation...")
times_large = np.linspace(0.1, 10.0, 100_000)

try:
    pdf_large = g.pdf_batch(times_large)
    print(f"Computed {len(pdf_large)} PDF values")
except Exception as e:
    print(f"Computation failed: {e}")

## Example 7: Multiple Operations

Monitor a sequence of different operations:

In [ ]:
%%usage

print("Phase 1: Matrix creation")
X = np.random.randn(1000, 1000)
time.sleep(0.5)

print("Phase 2: Matrix multiplication")
Y = X @ X.T
time.sleep(0.5)

print("Phase 3: Eigenvalue computation")
eigs = np.linalg.eigvals(Y[:300, :300])
time.sleep(0.5)

print("Phase 4: SVD")
U, s, Vh = np.linalg.svd(Y[:300, :300])

print("All phases complete!")

## Summary

### Key Features

- **Easy to use**: Just add `%%usage` to any cell
- **Auto-width**: Automatically fits notebook layout
- **Per-core monitoring**: See individual CPU core usage
- **Live updates**: Bars update in real-time during execution
- **Summary statistics**: Mean, min, max, and CPU-seconds
- **SLURM support**: Works on single-node and multi-node SLURM jobs

### Cell Magic Options

```python
%%usage                           # Default settings
%%usage --width 120               # Custom width
%%usage --interval 0.25           # Faster updates
%%usage --width 100 --interval 1.0  # Both custom
```

### Context Manager

```python
with pta.CPUMonitor():                           # Default
with pta.CPUMonitor(width=120):                  # Custom width
with pta.CPUMonitor(update_interval=0.25):       # Faster updates
with pta.CPUMonitor(show_summary=False):         # No summary
```

### SLURM Usage

The monitor automatically detects SLURM environment and shows:
- All allocated nodes
- CPUs per node
- Per-core usage for each node
- Summary statistics per node

Example SLURM script:
```bash
#!/bin/bash
#SBATCH --nodes=2
#SBATCH --cpus-per-task=8
#SBATCH --time=1:00:00

# Jupyter will show 2 nodes × 8 CPUs each
jupyter notebook cpu_monitoring_notebook.ipynb
```